In [1]:
#### for import clean data

import cv2
import os
import numpy as np
import pandas as pd
path = "C:/Users/Michael/Desktop/Berkeley/FinTech/DR/fintech_DR/images"
cleandata = pd.read_csv("C:/Users/Michael/Desktop/Berkeley/FinTech/DR/clean data.csv")
filenames = np.empty(1500)
filenames = cleandata["filename"]





In [2]:
#### read images and flat it

images = np.empty(len(filenames), dtype = object ) 
feature = []

for i in range(len(filenames)):              
    images[i] = cv2.imread(path + "/" + filenames[i])
    images[i] = cv2.resize(images[i], (512,512))
    #feature[i] = images[i].flatten()
    feature.extend(images[i])
feature_mat = np.matrix(np.reshape(feature, (1500,512*512*3)))
#feature_df = pd.DataFrame(feature)
feature_df = pd.DataFrame(feature_mat)




In [3]:
#### combining label and feature

fullset = pd.merge(cleandata, feature_df, left_index=True, right_index=True)
del fullset['Unnamed: 0']
fullset.head()


,filename,sight_threaten,0,1,2,3,4,5,6,7,...,786422,786423,786424,786425,786426,786427,786428,786429,786430,786431
0,36_right.jpeg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,57_left.jpeg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,60_left.jpeg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,78_right.jpeg,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,81_right.jpeg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
#### training and testing set split

from sklearn.model_selection import train_test_split 
from keras.utils import to_categorical
import random


random.seed( 30 )

y_train, y_test = train_test_split(fullset.iloc[:,0:2], test_size=0.2)



####################################

q = pd.DataFrame(y_train)
a = q[q["sight_threaten"].isin([0])]
b = q[q["sight_threaten"].isin([1])]


random.seed( 30 )
boota = np.random.choice(a["filename"], replace = True, size = 600)
bootb = np.random.choice(b["filename"], replace = True, size = 600)


### later use boot_df
boota_df = pd.DataFrame(boota, columns = ["filename"])
bootb_df = pd.DataFrame(bootb, columns = ["filename"])


train_filenames_df = pd.concat([boota_df, bootb_df])
y_new_train = pd.merge(train_filenames_df, cleandata.iloc[:,1:3], how = "left", on = ["filename", "filename"])
x_new_train = pd.merge(train_filenames_df, fullset, how = "left", on = ["filename", "filename"])
x_test = pd.merge(y_test.iloc[:,0:1], fullset, how = "left", on = ["filename", "filename"])

del x_new_train['sight_threaten']
del x_new_train['filename']
del x_test['sight_threaten']
del x_test['filename']
del fullset

#################################### 



# Onehot encoding for catogory
y_new_trainOneHot = to_categorical(y_new_train.iloc[:,1])
y_testOneHot = to_categorical(y_test.iloc[:,1])




Using TensorFlow backend.


In [5]:
x_test.head()

,0,1,2,3,4,5,6,7,8,9,...,786422,786423,786424,786425,786426,786427,786428,786429,786430,786431
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,3,1,2,3,1,3,4,2,3,...,0,1,2,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,1,3,3,3,4,4,4,3,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
#### PCA to do dimension reduction

from sklearn.decomposition import PCA
import time

n = 100
t0 = time.time()
pca = PCA(n_components=n, whiten=True).fit(x_new_train)
print("pca done %0.3fs" % (time.time() - t0))

print(sum(pca.explained_variance_ratio_))

t0 = time.time()
x_train_pca = pca.transform(x_new_train)
x_test_pca  = pca.transform(x_test)
del x_test
del x_new_train
print("data set to pca done %0.3fs" % (time.time() - t0))

pca done 740.206s
0.9844162163439683
data set to pca done 46.953s


In [7]:
### CNN 
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D




model = Sequential()


model.add(Dense(400, input_dim = n, activation = "relu"))

model.add(Dropout(0.4))


model.add(Dense(400, activation = "relu"))

model.add(Dropout(0.4))


model.add(Dense(400, activation = "relu"))

model.add(Dropout(0.4))

model.add(Dense(400, activation = "relu"))

model.add(Dropout(0.4))

model.add(Dense(400, activation = "relu"))

model.add(Dropout(0.4))

model.add(Dense(400, activation = "relu"))

model.add(Dropout(0.4))


model.add(Dense(400, activation = "relu"))

model.add(Dropout(0.4))

model.add(Dense(400, activation = "relu"))

model.add(Dropout(0.4))

model.add(Dense(400, activation = "relu"))

model.add(Dropout(0.4))



model.add(Dense(2, activation = 'softmax'))

model.compile(loss = "categorical_crossentropy",
             optimizer = "adam",
             metrics = ["accuracy"])
         

# maybe should change to focal loss?


model.summary()






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 400)               40400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 400)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 400)               160400    
_________________________________________________________________
dropout_2 (Dropout)          (None, 400)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 400)               160400    
_________________________________________________________________
dropout_3 (Dropout)          (None, 400)               0         
_____________________

In [8]:
random.seed( 30 )
model.fit(x = x_train_pca, y = y_new_trainOneHot, batch_size = 100, epochs = 1000, validation_split = 0.1, verbose = 2)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 1080 samples, validate on 120 samples
Epoch 1/1000
 - 3s - loss: 0.7362 - acc: 0.4944 - val_loss: 0.7622 - val_acc: 0.0000e+00
Epoch 2/1000
 - 0s - loss: 0.7110 - acc: 0.5083 - val_loss: 0.7565 - val_acc: 0.0000e+00
Epoch 3/1000
 - 0s - loss: 0.6959 - acc: 0.5509 - val_loss: 0.7055 - val_acc: 0.0000e+00
Epoch 4/1000
 - 0s - loss: 0.6957 - acc: 0.5194 - val_loss: 0.7238 - val_acc: 0.0000e+00
Epoch 5/1000
 - 0s - loss: 0.6931 - acc: 0.5380 - val_loss: 0.7211 - val_acc: 0.0000e+00
Epoch 6/1000
 - 0s - loss: 0.6896 - acc: 0.5296 - val_loss: 0.7408 - val_acc: 0.0000e+00
Epoch 7/1000
 - 0s - loss: 0.6879 - acc: 0.5574 - val_loss: 0.7389 - val_acc: 0.0000e+00
Epoch 8/1000
 - 0s - loss: 0.6891 - acc: 0.5556 - val_loss: 0.7231 - val_acc: 0.0000e+00
Epoch 9/1000
 - 0s - loss: 0.6796 - acc: 0.5593 - val_loss: 0.7258 - val_acc: 0.0000e+00
Epoch 10/1000
 - 0s - loss: 0.6716 - acc: 0.5537 - val_los

Epoch 92/1000
 - 0s - loss: 0.0149 - acc: 0.9944 - val_loss: 0.6061 - val_acc: 0.9083
Epoch 93/1000
 - 0s - loss: 0.0167 - acc: 0.9944 - val_loss: 0.7513 - val_acc: 0.9083
Epoch 94/1000
 - 0s - loss: 0.0054 - acc: 0.9981 - val_loss: 0.6787 - val_acc: 0.9083
Epoch 95/1000
 - 0s - loss: 0.0075 - acc: 0.9972 - val_loss: 0.9062 - val_acc: 0.9083
Epoch 96/1000
 - 0s - loss: 0.0105 - acc: 0.9954 - val_loss: 0.9015 - val_acc: 0.9083
Epoch 97/1000
 - 0s - loss: 0.0176 - acc: 0.9972 - val_loss: 0.7235 - val_acc: 0.9083
Epoch 98/1000
 - 0s - loss: 0.0094 - acc: 0.9972 - val_loss: 0.7899 - val_acc: 0.9083
Epoch 99/1000
 - 0s - loss: 0.0122 - acc: 0.9963 - val_loss: 0.8469 - val_acc: 0.9083
Epoch 100/1000
 - 0s - loss: 0.0061 - acc: 0.9981 - val_loss: 0.7011 - val_acc: 0.9083
Epoch 101/1000
 - 0s - loss: 0.0184 - acc: 0.9954 - val_loss: 0.9481 - val_acc: 0.9083
Epoch 102/1000
 - 0s - loss: 0.0254 - acc: 0.9907 - val_loss: 0.5506 - val_acc: 0.9167
Epoch 103/1000
 - 0s - loss: 0.0198 - acc: 0.9944 -

Epoch 187/1000
 - 0s - loss: 0.0050 - acc: 0.9991 - val_loss: 1.2287 - val_acc: 0.9083
Epoch 188/1000
 - 0s - loss: 0.0068 - acc: 0.9991 - val_loss: 1.1162 - val_acc: 0.9250
Epoch 189/1000
 - 0s - loss: 0.0058 - acc: 0.9981 - val_loss: 1.0288 - val_acc: 0.9250
Epoch 190/1000
 - 0s - loss: 0.0211 - acc: 0.9963 - val_loss: 1.0304 - val_acc: 0.9083
Epoch 191/1000
 - 0s - loss: 0.0115 - acc: 0.9981 - val_loss: 0.8854 - val_acc: 0.9083
Epoch 192/1000
 - 0s - loss: 0.0144 - acc: 0.9963 - val_loss: 0.6193 - val_acc: 0.9417
Epoch 193/1000
 - 0s - loss: 0.0046 - acc: 0.9981 - val_loss: 0.9630 - val_acc: 0.9083
Epoch 194/1000
 - 0s - loss: 0.0039 - acc: 0.9972 - val_loss: 0.9662 - val_acc: 0.9083
Epoch 195/1000
 - 0s - loss: 7.7314e-04 - acc: 1.0000 - val_loss: 0.9883 - val_acc: 0.9083
Epoch 196/1000
 - 0s - loss: 0.0028 - acc: 0.9991 - val_loss: 1.0123 - val_acc: 0.9083
Epoch 197/1000
 - 0s - loss: 0.0154 - acc: 0.9963 - val_loss: 0.9959 - val_acc: 0.9250
Epoch 198/1000
 - 0s - loss: 0.0074 - a

Epoch 281/1000
 - 0s - loss: 0.0215 - acc: 0.9935 - val_loss: 1.1747 - val_acc: 0.8917
Epoch 282/1000
 - 0s - loss: 0.0144 - acc: 0.9972 - val_loss: 0.8035 - val_acc: 0.9167
Epoch 283/1000
 - 0s - loss: 0.0316 - acc: 0.9907 - val_loss: 0.8841 - val_acc: 0.9083
Epoch 284/1000
 - 0s - loss: 0.0119 - acc: 0.9944 - val_loss: 1.2106 - val_acc: 0.8917
Epoch 285/1000
 - 0s - loss: 0.0050 - acc: 0.9963 - val_loss: 0.9874 - val_acc: 0.9083
Epoch 286/1000
 - 0s - loss: 0.0063 - acc: 0.9972 - val_loss: 1.0346 - val_acc: 0.8917
Epoch 287/1000
 - 0s - loss: 0.0041 - acc: 0.9972 - val_loss: 1.1383 - val_acc: 0.8917
Epoch 288/1000
 - 0s - loss: 0.0057 - acc: 0.9991 - val_loss: 1.0572 - val_acc: 0.8917
Epoch 289/1000
 - 0s - loss: 0.0049 - acc: 0.9991 - val_loss: 0.9665 - val_acc: 0.8917
Epoch 290/1000
 - 0s - loss: 0.0066 - acc: 0.9981 - val_loss: 1.0715 - val_acc: 0.9000
Epoch 291/1000
 - 0s - loss: 0.0183 - acc: 0.9963 - val_loss: 0.5432 - val_acc: 0.9333
Epoch 292/1000
 - 0s - loss: 0.0220 - acc: 

 - 0s - loss: 0.0252 - acc: 0.9935 - val_loss: 0.9336 - val_acc: 0.9333
Epoch 376/1000
 - 0s - loss: 0.0177 - acc: 0.9944 - val_loss: 0.8620 - val_acc: 0.9333
Epoch 377/1000
 - 0s - loss: 0.0111 - acc: 0.9963 - val_loss: 1.0777 - val_acc: 0.9333
Epoch 378/1000
 - 0s - loss: 0.0051 - acc: 0.9981 - val_loss: 1.3339 - val_acc: 0.9000
Epoch 379/1000
 - 0s - loss: 0.0081 - acc: 0.9972 - val_loss: 1.2908 - val_acc: 0.8917
Epoch 380/1000
 - 0s - loss: 0.0130 - acc: 0.9972 - val_loss: 1.1880 - val_acc: 0.9167
Epoch 381/1000
 - 0s - loss: 0.0336 - acc: 0.9963 - val_loss: 0.7789 - val_acc: 0.9167
Epoch 382/1000
 - 0s - loss: 0.0077 - acc: 0.9981 - val_loss: 1.1753 - val_acc: 0.9000
Epoch 383/1000
 - 0s - loss: 0.0045 - acc: 0.9991 - val_loss: 1.2264 - val_acc: 0.9000
Epoch 384/1000
 - 0s - loss: 0.0070 - acc: 0.9963 - val_loss: 1.1341 - val_acc: 0.9000
Epoch 385/1000
 - 0s - loss: 0.0058 - acc: 0.9981 - val_loss: 0.9585 - val_acc: 0.9333
Epoch 386/1000
 - 0s - loss: 0.0089 - acc: 0.9981 - val_lo

...

Epoch 564/1000
 - 0s - loss: 0.0103 - acc: 0.9972 - val_loss: 0.9656 - val_acc: 0.9250
Epoch 565/1000
 - 0s - loss: 0.0063 - acc: 0.9991 - val_loss: 0.9068 - val_acc: 0.9333
Epoch 566/1000
 - 0s - loss: 0.0113 - acc: 0.9954 - val_loss: 0.8701 - val_acc: 0.9417
Epoch 567/1000
 - 0s - loss: 0.0032 - acc: 0.9981 - val_loss: 0.9463 - val_acc: 0.9417
Epoch 568/1000
 - 0s - loss: 0.0192 - acc: 0.9981 - val_loss: 0.9562 - val_acc: 0.9250
Epoch 569/1000
 - 0s - loss: 0.0061 - acc: 0.9972 - val_loss: 0.9041 - val_acc: 0.9250
Epoch 570/1000
 - 0s - loss: 0.0073 - acc: 0.9991 - val_loss: 1.2202 - val_acc: 0.9083
Epoch 571/1000
 - 0s - loss: 0.0142 - acc: 0.9935 - val_loss: 1.0937 - val_acc: 0.9167
Epoch 572/1000
 - 0s - loss: 0.0060 - acc: 0.9981 - val_loss: 1.0379 - val_acc: 0.9333
Epoch 573/1000
 - 0s - loss: 0.0143 - acc: 0.9972 - val_loss: 1.0805 - val_acc: 0.9333
Epoch 574/1000
 - 0s - loss: 0.0041 - acc: 0.9981 - val_loss: 1.1985 - val_acc: 0.9250
Epoch 575/1000
 - 0s - loss: 0.0042 - acc: 

Epoch 658/1000
 - 0s - loss: 0.0127 - acc: 0.9963 - val_loss: 1.0298 - val_acc: 0.9250
Epoch 659/1000
 - 0s - loss: 0.0162 - acc: 0.9981 - val_loss: 1.0146 - val_acc: 0.9250
Epoch 660/1000
 - 0s - loss: 0.0073 - acc: 0.9963 - val_loss: 1.2138 - val_acc: 0.9083
Epoch 661/1000
 - 0s - loss: 7.1236e-04 - acc: 1.0000 - val_loss: 1.2013 - val_acc: 0.9083
Epoch 662/1000
 - 0s - loss: 0.0024 - acc: 0.9991 - val_loss: 1.1670 - val_acc: 0.9083
Epoch 663/1000
 - 0s - loss: 2.7521e-05 - acc: 1.0000 - val_loss: 1.1299 - val_acc: 0.9250
Epoch 664/1000
 - 0s - loss: 0.0184 - acc: 0.9981 - val_loss: 1.0840 - val_acc: 0.9250
Epoch 665/1000
 - 0s - loss: 0.0403 - acc: 0.9972 - val_loss: 1.0269 - val_acc: 0.9250
Epoch 666/1000
 - 0s - loss: 0.0373 - acc: 0.9954 - val_loss: 1.0973 - val_acc: 0.9083
Epoch 667/1000
 - 0s - loss: 0.0136 - acc: 0.9981 - val_loss: 1.1100 - val_acc: 0.9083
Epoch 668/1000
 - 0s - loss: 0.0035 - acc: 1.0000 - val_loss: 1.0810 - val_acc: 0.9083
Epoch 669/1000
 - 0s - loss: 0.0321

Epoch 752/1000
 - 0s - loss: 8.6394e-04 - acc: 0.9991 - val_loss: 1.0205 - val_acc: 0.9250
Epoch 753/1000
 - 0s - loss: 0.0138 - acc: 0.9963 - val_loss: 0.9648 - val_acc: 0.9250
Epoch 754/1000
 - 0s - loss: 0.0340 - acc: 0.9972 - val_loss: 0.7453 - val_acc: 0.9417
Epoch 755/1000
 - 0s - loss: 0.0275 - acc: 0.9972 - val_loss: 0.8432 - val_acc: 0.9417
Epoch 756/1000
 - 0s - loss: 9.5459e-04 - acc: 1.0000 - val_loss: 0.9477 - val_acc: 0.9250
Epoch 757/1000
 - 0s - loss: 0.0023 - acc: 0.9981 - val_loss: 0.9714 - val_acc: 0.9250
Epoch 758/1000
 - 0s - loss: 0.0028 - acc: 0.9991 - val_loss: 0.9550 - val_acc: 0.9333
Epoch 759/1000
 - 0s - loss: 0.0014 - acc: 0.9991 - val_loss: 0.9540 - val_acc: 0.9333
Epoch 760/1000
 - 0s - loss: 0.0144 - acc: 0.9981 - val_loss: 0.9235 - val_acc: 0.9333
Epoch 761/1000
 - 0s - loss: 0.0159 - acc: 0.9981 - val_loss: 0.8975 - val_acc: 0.9417
Epoch 762/1000
 - 0s - loss: 9.5229e-04 - acc: 1.0000 - val_loss: 0.9485 - val_acc: 0.9333
Epoch 763/1000
 - 0s - loss: 0.

Epoch 846/1000
 - 0s - loss: 0.0137 - acc: 0.9972 - val_loss: 0.8999 - val_acc: 0.9333
Epoch 847/1000
 - 0s - loss: 0.0010 - acc: 1.0000 - val_loss: 0.8272 - val_acc: 0.9333
Epoch 848/1000
 - 0s - loss: 0.0045 - acc: 0.9972 - val_loss: 0.8221 - val_acc: 0.9333
Epoch 849/1000
 - 0s - loss: 0.0343 - acc: 0.9963 - val_loss: 0.8361 - val_acc: 0.9333
Epoch 850/1000
 - 0s - loss: 0.0323 - acc: 0.9981 - val_loss: 0.9621 - val_acc: 0.9333
Epoch 851/1000
 - 0s - loss: 0.0154 - acc: 0.9991 - val_loss: 1.0443 - val_acc: 0.9333
Epoch 852/1000
 - 0s - loss: 0.0028 - acc: 0.9981 - val_loss: 1.0624 - val_acc: 0.9333
Epoch 853/1000
 - 0s - loss: 0.0032 - acc: 0.9991 - val_loss: 1.0791 - val_acc: 0.9333
Epoch 854/1000
 - 0s - loss: 0.0175 - acc: 0.9981 - val_loss: 1.2895 - val_acc: 0.9167
Epoch 855/1000
 - 0s - loss: 6.5227e-04 - acc: 1.0000 - val_loss: 1.3486 - val_acc: 0.9167
Epoch 856/1000
 - 0s - loss: 0.0030 - acc: 0.9981 - val_loss: 1.2727 - val_acc: 0.9167
Epoch 857/1000
 - 0s - loss: 2.8910e-04

Epoch 939/1000
 - 0s - loss: 0.0165 - acc: 0.9981 - val_loss: 1.1829 - val_acc: 0.9167
Epoch 940/1000
 - 0s - loss: 4.6951e-04 - acc: 1.0000 - val_loss: 1.2515 - val_acc: 0.9167
Epoch 941/1000
 - 0s - loss: 2.1535e-07 - acc: 1.0000 - val_loss: 1.2708 - val_acc: 0.9167
Epoch 942/1000
 - 0s - loss: 7.2339e-04 - acc: 0.9991 - val_loss: 1.2433 - val_acc: 0.9167
Epoch 943/1000
 - 0s - loss: 2.1755e-04 - acc: 1.0000 - val_loss: 1.1598 - val_acc: 0.9167
Epoch 944/1000
 - 0s - loss: 4.4122e-04 - acc: 1.0000 - val_loss: 1.2104 - val_acc: 0.9167
Epoch 945/1000
 - 0s - loss: 0.0152 - acc: 0.9991 - val_loss: 1.2769 - val_acc: 0.9167
Epoch 946/1000
 - 0s - loss: 2.0850e-05 - acc: 1.0000 - val_loss: 1.3047 - val_acc: 0.9167
Epoch 947/1000
 - 0s - loss: 0.0010 - acc: 0.9991 - val_loss: 1.3432 - val_acc: 0.9167
Epoch 948/1000
 - 0s - loss: 0.0298 - acc: 0.9981 - val_loss: 1.3432 - val_acc: 0.9167
Epoch 949/1000
 - 0s - loss: 1.3074e-07 - acc: 1.0000 - val_loss: 1.3432 - val_acc: 0.9167
Epoch 950/1000


In [9]:
scores = model.evaluate(x_test_pca, y_testOneHot)
scores[1]

300/300 [==============================] - 0s 103us/step


0.6733333333333333

In [10]:

prediction = model.predict_classes(x_test_pca)
pd.crosstab(y_test.iloc[:,1],prediction,
            rownames=['True'],colnames=['Predict'])

Predict,0,1
True,,
0,174,70
1,28,28


In [11]:
#### confusion table

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


names = ["Healthy eyes", "Dr Disease"]
print(classification_report(y_test.iloc[:,1], prediction, target_names = names))
#print(confusion_matrix(y_test, prediction, labels=[0,1]))


              precision    recall  f1-score   support

Healthy eyes       0.86      0.71      0.78       244
  Dr Disease       0.29      0.50      0.36        56

    accuracy                           0.67       300
   macro avg       0.57      0.61      0.57       300
weighted avg       0.75      0.67      0.70       300

